In [2]:
%load_ext autoreload
%autoreload 2

In [20]:
import yaml
import os
import urlparse
import requests
import json
import time

from synorchestrator.trs_client.client import TRSClient
from synorchestrator.wes_client.client import WESClient
from synorchestrator.util import build_wes_request
from synorchestrator.util import params_url2object

## Set up TRS and WES clients

Load options and paramters for TRS and WES servers from local configs.

In [4]:
with open('../toolregistries.config', 'rb') as f:
    trs_config = yaml.load(f)
    
with open('../workflowservices.config', 'rb') as f:
    wes_config = yaml.load(f)

with open('../evals.config', 'rb') as f:
    eval_config = yaml.load(f)

In [5]:
tclient = TRSClient(**trs_config['dockstore'])
arvclient = WESClient(**wes_config['arvados-wes'])
cromclient = WESClient(**wes_config['hca-cromwell'])

## Tool test

Using this section to do some lightweight testing with standalone WDL/CWL *tools* (before getting into workflows).

### WDL

Testing with the WDL version of `quay.io/briandoconnor/dockstore-tool-md5sum`; fetch the descriptor file content from Dockstore.

In [10]:
wdl_tool_url = 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/plain-WDL/descriptor/%2FDockstore.wdl'
wdl_tool_descriptor = requests.get(wdl_tool_url)
wdl_tool_descriptor.content

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/plain-WDL/descriptor/%2FDockstore.wdl HTTP/1.1" 200 211


'task md5 {\n  File inputFile\n\n  command {\n    /bin/my_md5sum ${inputFile}\n  }\n\n output {\n    File value = "md5sum.txt"\n }\n\n runtime {\n   docker: "quay.io/briandoconnor/dockstore-tool-md5sum:1.0.4"\n   cpu: 1\n   memory: "512 MB"\n }\n}\n\nworkflow ga4ghMd5 {\n File inputFile\n call md5 { input: inputFile=inputFile }\n}\n'

#### Submitting with `workflow_descriptor`

Retrieve the test parameters content from the JSON file in GitHub and format as an object for the WES request. Provide the contents of WDL descriptor file as string/blob in request.

In [11]:
wdl_tool_test_url = 'https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.wdl.json'
tool_request = build_wes_request(
    workflow_descriptor=wdl_tool_descriptor.content, 
    workflow_params=wdl_tool_test_url,
    workflow_type='WDL'
    
)
tool_request

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.wdl.json" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 50786
DEBUG:cachecontrol.controller:Freshness lifetime from max-age: 300
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /briandoconnor/dockstore-tool-md5sum/master/test.wdl.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /briandoconnor/dockstore-tool-md5sum/master/md5sum.input HTTP/1.1" 200 0


{'workflow_descriptor': 'task md5 {\n  File inputFile\n\n  command {\n    /bin/my_md5sum ${inputFile}\n  }\n\n output {\n    File value = "md5sum.txt"\n }\n\n runtime {\n   docker: "quay.io/briandoconnor/dockstore-tool-md5sum:1.0.4"\n   cpu: 1\n   memory: "512 MB"\n }\n}\n\nworkflow ga4ghMd5 {\n File inputFile\n call md5 { input: inputFile=inputFile }\n}\n',
 'workflow_params': ordereddict([('ga4ghMd5.inputFile', 'https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/md5sum.input')]),
 'workflow_type': 'WDL',
 'workflow_type_version': 'v1.0'}

Update the file input path in `workflow_params` to use GS URL.

In [327]:
tool_request['workflow_params']['ga4ghMd5.inputFile'] = 'gs://dockstore-tool-md5sum-data/md5sum.input.txt'
tool_request

{'workflow_descriptor': 'task md5 {\n  File inputFile\n\n  command {\n    /bin/my_md5sum ${inputFile}\n  }\n\n output {\n    File value = "md5sum.txt"\n }\n\n runtime {\n   docker: "quay.io/briandoconnor/dockstore-tool-md5sum:1.0.4"\n   cpu: 1\n   memory: "512 MB"\n }\n}\n\nworkflow ga4ghMd5 {\n File inputFile\n call md5 { input: inputFile=inputFile }\n}\n',
 'workflow_params': ordereddict([('ga4ghMd5.inputFile', 'gs://dockstore-tool-md5sum-data/md5sum.input.txt')]),
 'workflow_type': 'WDL',
 'workflow_type_version': 'v1.0'}

Submit the tool run.

In [328]:
cromclient.run_workflow(tool_request)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): g0n2qjnu94.execute-api.us-east-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://g0n2qjnu94.execute-api.us-east-1.amazonaws.com:443 "POST /test//workflows HTTP/1.1" 200 55


{u'workflow_id': u'0711d724-afbe-4867-ab4b-e415468b2be5'}

In [334]:
cromclient.get_workflow_run('0711d724-afbe-4867-ab4b-e415468b2be5')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): g0n2qjnu94.execute-api.us-east-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://g0n2qjnu94.execute-api.us-east-1.amazonaws.com:443 "GET /test//workflows/0711d724-afbe-4867-ab4b-e415468b2be5 HTTP/1.1" 200 962


{u'end_time': u'2018-05-24T02:18:17.323Z',
 u'outputs': {u'ga4ghMd5.md5.value': u'gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/ga4ghMd5/0711d724-afbe-4867-ab4b-e415468b2be5/call-md5/md5sum.txt'},
 u'request': {u'workflow_descriptor': u'task md5 {\n  File inputFile\n\n  command {\n    /bin/my_md5sum ${inputFile}\n  }\n\n output {\n    File value = "md5sum.txt"\n }\n\n runtime {\n   docker: "quay.io/briandoconnor/dockstore-tool-md5sum:1.0.4"\n   cpu: 1\n   memory: "512 MB"\n }\n}\n\nworkflow ga4ghMd5 {\n File inputFile\n call md5 { input: inputFile=inputFile }\n}\n',
  u'workflow_params': {u'ga4ghMd5.inputFile': u'gs://dockstore-tool-md5sum-data/md5sum.input.txt'},
  u'workflow_type': u'WDL'},
 u'start_time': u'2018-05-24T02:16:19.746Z',
 u'state': u'COMPLETE',
 u'task_logs': [],
 u'workflow_id': u'0711d724-afbe-4867-ab4b-e415468b2be5',
 u'workflow_log': {u'cmd': [u''],
  u'end_time': u'',
  u'exit_code': 0,
  u'name': u'',
  u'start_time': u'',
  u'stderr': u'',
  u's

#### Submitting with `workflow_url`

Attempt to run the tool by providing URL path for the WDL descriptor file in WES request.

In [339]:
tool_request = build_wes_request(
    workflow_url='https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/plain-WDL/descriptor/%2FDockstore.wdl', 
    workflow_params='https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.wdl.json',
    workflow_type='WDL'
    
)
tool_request['workflow_params']['ga4ghMd5.inputFile'] = 'gs://dockstore-tool-md5sum-data/md5sum.input.txt'
tool_request

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.wdl.json" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 211
DEBUG:cachecontrol.controller:Freshness lifetime from max-age: 300
DEBUG:cachecontrol.controller:The response is "fresh", returning cached response
DEBUG:cachecontrol.controller:300 > 211
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /briandoconnor/dockstore-tool-md5sum/master/md5sum.input HTTP/1.1" 200 0


{'workflow_params': ordereddict([('ga4ghMd5.inputFile', 'gs://dockstore-tool-md5sum-data/md5sum.input.txt')]),
 'workflow_type': 'WDL',
 'workflow_type_version': 'v1.0',
 'workflow_url': 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/plain-WDL/descriptor/%2FDockstore.wdl'}

In [340]:
cromclient.run_workflow(tool_request)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): g0n2qjnu94.execute-api.us-east-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://g0n2qjnu94.execute-api.us-east-1.amazonaws.com:443 "POST /test//workflows HTTP/1.1" 200 232


{u'errorMessage': u"'workflow_descriptor'",
 u'errorType': u'KeyError',
 u'stackTrace': [[u'/var/task/wes_to_cromwell.py',
   93,
   u'wes_workflows_post',
   u'z = zipfile.ZipFile(io.BytesIO(base64.b64decode(event["workflow_descriptor"])), \'r\')']]}

### CWL

Testing with the CWL version of `quay.io/briandoconnor/dockstore-tool-md5sum`; fetch the descriptor content from Dockstore.

In [309]:
cwl_tool_url = 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/CWL/descriptor'
cwl_tool_descriptor = requests.get(cwl_tool_url).json()
cwl_tool_descriptor

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/CWL/descriptor HTTP/1.1" 200 775


{u'descriptor': u'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: Md5sum\nlabel: Simple md5sum tool\ncwlVersion: v1.0\n\n$namespaces:\n  dct: http://purl.org/dc/terms/\n  foaf: http://xmlns.com/foaf/0.1/\n\ndoc: |\n  [![Docker Repository on Quay.io](https://quay.io/repository/briandoconnor/dockstore-tool-md5sum/status "Docker Repository on Quay.io")](https://quay.io/repository/briandoconnor/dockstore-tool-md5sum)\n  [![Build Status](https://travis-ci.org/briandoconnor/dockstore-tool-md5sum.svg)](https://travis-ci.org/briandoconnor/dockstore-tool-md5sum)\n  A very, very simple Docker container for the md5sum command. See the [README](https://github.com/briandoconnor/dockstore-tool-md5sum/blob/master/README.md) for more information.\n\n\n#dct:creator:\n#  \'@id\': http://orcid.org/0000-0002-7681-6415\n#  foaf:name: Brian O\'Connor\n#  foaf:mbox: briandoconnor@gmail.com\n\nrequirements:\n- class: DockerRequirement\n  dockerPull: quay.io/briandoconnor/dockstore-tool-md5sum:1.0.4\n

#### Submitting with `workflow_descriptor`

Retrieve the test parameters content from the JSON file in GitHub and format as an object for the WES request. Provide the contents of CWL descriptor file as string/blob in request.

In [310]:
cwl_tool_request = build_wes_request(
    workflow_descriptor=cwl_tool_descriptor['descriptor'], 
#     workflow_url=cwl_tool_descriptor['url'],
    workflow_params='https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.json',
    workflow_type='CWL'
)
cwl_tool_request

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/briandoconnor/dockstore-tool-md5sum/master/test.json" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 545
DEBUG:cachecontrol.controller:Freshness lifetime from max-age: 300
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /briandoconnor/dockstore-tool-md5sum/master/test.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /briandoconnor/dockstore-tool-md5sum/master/md5sum.input HTTP/1.1" 200 0


{'workflow_descriptor': u'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: Md5sum\nlabel: Simple md5sum tool\ncwlVersion: v1.0\n\n$namespaces:\n  dct: http://purl.org/dc/terms/\n  foaf: http://xmlns.com/foaf/0.1/\n\ndoc: |\n  [![Docker Repository on Quay.io](https://quay.io/repository/briandoconnor/dockstore-tool-md5sum/status "Docker Repository on Quay.io")](https://quay.io/repository/briandoconnor/dockstore-tool-md5sum)\n  [![Build Status](https://travis-ci.org/briandoconnor/dockstore-tool-md5sum.svg)](https://travis-ci.org/briandoconnor/dockstore-tool-md5sum)\n  A very, very simple Docker container for the md5sum command. See the [README](https://github.com/briandoconnor/dockstore-tool-md5sum/blob/master/README.md) for more information.\n\n\n#dct:creator:\n#  \'@id\': http://orcid.org/0000-0002-7681-6415\n#  foaf:name: Brian O\'Connor\n#  foaf:mbox: briandoconnor@gmail.com\n\nrequirements:\n- class: DockerRequirement\n  dockerPull: quay.io/briandoconnor/dockstore-tool-md5sum

In [311]:
arvclient.run_workflow(cwl_tool_request)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "POST /ga4gh/wes/v1/workflows HTTP/1.1" 200 51


{u'workflow_id': u'qr1hi-xvhdp-y88n0ox6fnvaswt'}

In [12]:
arvclient.get_workflow_run('qr1hi-xvhdp-y88n0ox6fnvaswt')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-y88n0ox6fnvaswt HTTP/1.1" 200 254


{u'outputs': {},
 u'request': {},
 u'state': u'QUEUED',
 u'task_logs': [],
 u'workflow_id': u'qr1hi-xvhdp-y88n0ox6fnvaswt',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'startTime': u'',
  u'stderr': u'',
  u'stdout': u''}}

## Working with TRS

### Get checker workflow information

In [13]:
workflow_id = eval_config[0]['workflow_id']
workflow_id

'github.com/dockstore-testing/md5sum-checker'

#### Get checker for specified workflow

In [14]:
checker_workflow = tclient.get_workflow_checker(workflow_id)
checker_workflow['id']

DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker HTTP/1.1" 200 418
DEBUG:synorchestrator.trs_client.client:found checker workflow: github.com/dockstore-testing/md5sum-checker/_cwl_checker
DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker HTTP/1.1" 200 496


u'#workflow/github.com/dockstore-testing/md5sum-checker/_cwl_checker'

#### Get checker descriptor for specified version and workflow type

In [15]:
checker_descriptor = tclient.get_workflow_descriptor(checker_workflow['id'], 'develop', 'CWL')
checker_descriptor

DEBUG:synorchestrator.trs_client.client:getting descriptor from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor HTTP/1.1" 200 541


{u'descriptor': u'cwlVersion: v1.0\nclass: Workflow\n\nrequirements:\n  - class: SubworkflowFeatureRequirement\n\n#dct:creator:\n#  \'@id\': http://orcid.org/0000-0002-7681-6415\n#  foaf:name: Brian O\'Connor\n#  foaf:mbox: mailto:briandoconnor@gmail.com\n\n#dct:contributor:\n#  foaf:name: Denis Yuen\n#  foaf:mbox: mailto:denis.yuen@oicr.on.ca\n\ninputs:\n  input_file: File\n  expected_md5: string\n\noutputs:\n  workflow_output_file:\n    type: File\n    outputSource: checker/results_file\n\nsteps:\n  md5sum:\n    run: md5sum/md5sum-workflow.cwl\n    in:\n      input_file: input_file\n    out: [output_file]\n  checker:\n    run: checker/md5sum-checker.cwl\n    in:\n      input_file: md5sum/output_file\n      expected_md5: expected_md5\n    out: [results_file]\n\ndoc: |\n  This demonstrates how to wrap a "real" tool with a checker workflow that runs both the tool and a tool that performs verification of results\n',
 u'type': u'CWL',
 u'url': u'https://raw.githubusercontent.com/dockstore

#### Get secondary descriptor(s) for checker workflow

#### Get checker test params for specified version and workflow type

In [16]:
checker_tests = tclient.get_workflow_tests(checker_workflow['id'], 'develop', 'CWL')
checker_tests

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/tests HTTP/1.1" 200 206
DEBUG:synorchestrator.trs_client.client:getting descriptor from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor HTTP/1.1" 200 541


[{u'test': u'{\n  "input_file": {\n        "class": "File",\n        "path": "md5sum.input"\n    },\n    "expected_md5": "00579a00e3e7fa0674428ac7049423e2"\n}\n',
  u'url': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json'}]

## Prepare workflow run request

### Build request

In [17]:
workflow_request = build_wes_request(
    checker_descriptor['url'], checker_tests[0]['url']
)
workflow_request

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 165907
DEBUG:cachecontrol.controller:Freshness lifetime from max-age: 300
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl HTTP/1.1" 304 0


{'workflow_descriptor': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json',
 'workflow_params': ordereddict([('cwlVersion', 'v1.0'), ('class', 'Workflow'), ('requirements', [ordereddict([('class', 'SubworkflowFeatureRequirement')])]), ('inputs', ordereddict([('input_file', 'File'), ('expected_md5', 'string')])), ('outputs', ordereddict([('workflow_output_file', ordereddict([('type', 'File'), ('outputSource', 'checker/results_file')]))])), ('steps', ordereddict([('md5sum', ordereddict([('run', 'md5sum/md5sum-workflow.cwl'), ('in', ordereddict([('input_file', 'input_file')])), ('out', ['output_file'])])), ('checker', ordereddict([('run', 'checker/md5sum-checker.cwl'), ('in', ordereddict([('input_file', 'md5sum/output_file'), ('expected_md5', 'expected_md5')])), ('out', ['results_file'])]))])), ('doc', u'This demonstrates how to wrap a "real" tool with a checker workflow that runs both the tool and a tool that performs verification of resu

## Working with WES

### Verify WES connections are working

In [19]:
arvclient.get_service_info()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/service-info HTTP/1.1" 200 325


{u'engine_versions': u'cwl-runner',
 u'key_values': {},
 u'supported_filesystem_protocols': [u'file', u'http', u'https', u'keep'],
 u'supported_wes_versions': u'0.2.1',
 u'system_state_counts': {},
 u'workflow_type_versions': {u'CWL': {u'workflow_type_version': [u'v1.0']}}}

In [103]:
cromclient.get_service_info()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): g0n2qjnu94.execute-api.us-east-1.amazonaws.com
DEBUG:urllib3.connectionpool:https://g0n2qjnu94.execute-api.us-east-1.amazonaws.com:443 "GET /test//service-info HTTP/1.1" 200 337


{u'auth_instructions_url': u'',
 u'default_workflow_engine_parameters': [],
 u'supported_filesystem_protocols': [u'gs'],
 u'supported_wes_versions': [u'0.2.1'],
 u'system_state_counts': {u'zero': 0},
 u'tags': {},
 u'workflow_engine_versions': {u'cromwell': u'31'},
 u'workflow_type_versions': {u'huh': {u'worfklow_type_version': [u'wdl',
    u'gzip',
    u'main.wdl']}}}

### Run a checker workflow

#### Submit new workflow run

In [18]:
r = arvclient.run_workflow(workflow_request)
r

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "POST /ga4gh/wes/v1/workflows HTTP/1.1" 200 51


{u'workflow_id': u'qr1hi-xvhdp-9frmmblu8vo749f'}

#### Monitor workflow run status

In [22]:
r = arvclient.get_workflow_run_status(r['workflow_id'])
r
# while r['state'] in ('QUEUED', 'INITIALIZING', 'RUNNING'):
#     time.sleep(1)
#     r = arvclient.get_workflow_run_status(r['workflow_id'])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-9frmmblu8vo749f/status HTTP/1.1" 200 72


{u'state': u'QUEUED', u'workflow_id': u'qr1hi-xvhdp-9frmmblu8vo749f'}

#### Check workflow run details

In [23]:
arvclient.get_workflow_run(r['workflow_id'])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-9frmmblu8vo749f HTTP/1.1" 200 254


{u'outputs': {},
 u'request': {},
 u'state': u'QUEUED',
 u'task_logs': [],
 u'workflow_id': u'qr1hi-xvhdp-9frmmblu8vo749f',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'startTime': u'',
  u'stderr': u'',
  u'stdout': u''}}